[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pachterlab/Clytia_single_cell_2020/blob/master/notebooks/cellAtlas_Generation.ipynb)

In [1]:
!date

Wed Jun  3 20:54:09 UTC 2020


In [2]:
#Download starvation data from Box, raw Kallisto counts, no clustering, etc
!wget https://caltech.box.com/shared/static/3l60szs0e9hgaax4q9dbuehgoo092xww

--2020-06-03 20:54:14--  https://caltech.box.com/shared/static/3l60szs0e9hgaax4q9dbuehgoo092xww
Resolving caltech.box.com (caltech.box.com)... 107.152.26.197
Connecting to caltech.box.com (caltech.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/3l60szs0e9hgaax4q9dbuehgoo092xww [following]
--2020-06-03 20:54:14--  https://caltech.box.com/public/static/3l60szs0e9hgaax4q9dbuehgoo092xww
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/3l60szs0e9hgaax4q9dbuehgoo092xww [following]
--2020-06-03 20:54:14--  https://caltech.app.box.com/public/static/3l60szs0e9hgaax4q9dbuehgoo092xww
Resolving caltech.app.box.com (caltech.app.box.com)... 107.152.29.201
Connecting to caltech.app.box.com (caltech.app.box.com)|107.152.29.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: h

In [3]:
#Download stimulation data from Box, raw Kallisto counts, no clustering, etc
!wget https://caltech.box.com/shared/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45
!wget https://caltech.box.com/shared/static/hd36giuxz3zd933fh4bca3iqnykncro8

--2020-06-03 20:54:21--  https://caltech.box.com/shared/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45
Resolving caltech.box.com (caltech.box.com)... 107.152.26.197
Connecting to caltech.box.com (caltech.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45 [following]
--2020-06-03 20:54:21--  https://caltech.box.com/public/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45 [following]
--2020-06-03 20:54:21--  https://caltech.app.box.com/public/static/cy5n9b6rdyk7v4uhvgvpkgotmchlts45
Resolving caltech.app.box.com (caltech.app.box.com)... 107.152.29.201
Connecting to caltech.app.box.com (caltech.app.box.com)|107.152.29.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: h

In [0]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [4]:
!pip install scvi

     |████████████████████████████████| 163kB 2.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 8.7MB 6.1MB/s 
     |████████████████████████████████| 112kB 41.5MB/s 
     |████████████████████████████████| 122kB 47.9MB/s 
     |████████████████████████████████| 7.2MB 16.6MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for loompy: filename=loompy-3.0.6-cp36-none-any.whl size=47895 sha256=e542092a289aed9ba70a9ada759188969c390e4f18c5259462ad547d2944f986
  Stored in directory: /root/.cache/pip/wheels/f9/a4/90/5a98ad83419732b0fba533b81a2a52ba3dbe230a936ca4cdc9
  Created wheel for numpy-groupies: filename=numpy_groupies-0+unknown-cp36-none-any.whl size=28042 sha256=ecacca816fcd47ad6b87176986ea90fafbde990cd628c8dbadb10b9b71e66cb0
  Stored in directory: /root/.cache/pip/wheels/30/ac/83/64d5f9293aeaec63f9539142fc629a41af064cae1b3d8d94aa
Successfully built loomp

**Install packages**

In [0]:
import torch
from scvi.dataset import AnnDatasetFromAnnData, GeneExpressionDataset, SmfishDataset
from scvi.models import VAE, LDVAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed

import pandas as pd
import anndata
import scanpy as sc
import numpy as np
import scipy.sparse

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
sc.set_figure_params(dpi=125)

import seaborn as sns
sns.set(style="whitegrid")

##**Generation of cell atlas**
### Using scVI (LDVAE) for merging of datasets (Lopez et al. 2018)

Read in data

In [7]:
#Stimulation data
#path = './drive/My Drive/Colab Notebooks/data/stimulation'

#Raw counts, no clustering h5ad, from Kallisto bus
jelly4stim = anndata.read("cy5n9b6rdyk7v4uhvgvpkgotmchlts45")

#Kallisto bus h5ad with clustering and filtering 
bus_stim = anndata.read("hd36giuxz3zd933fh4bca3iqnykncro8")

#Remove Cluster '0' cells (low umi counts and gene expression)
jelly4stim = jelly4stim[bus_stim.obs_names,]


#Add _ieg to cell names
jelly4stim.obs_names = [i+'_ieg' for i in jelly4stim.obs_names]


stim_data = AnnDatasetFromAnnData(ad = jelly4stim)

#Filter zero count genes
stim_data.filter_genes_by_count(per_batch=True)
print(stim_data)



#Read in Fed/Starved data, raw counts, no clustering
oj2 = anndata.read("3l60szs0e9hgaax4q9dbuehgoo092xww")

#Add _fs to cell names
oj2.obs_names = [i+'_fs' for i in oj2.obs_names]


fs_data = AnnDatasetFromAnnData(ad = oj2)

#Filter zero count genes
fs_data.filter_genes_by_count(per_batch=True)
print(fs_data)


#Concatenate data (with all nonzero genes)
combo = jelly4stim.concatenate(oj2,join='outer', index_unique=None)

[2020-06-03 20:56:08,010] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-03 20:56:08,014] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-03 20:56:08,557] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-03 20:56:08,673] INFO - scvi.dataset.dataset | Downsampled from 19054 to 19054 cells
[2020-06-03 20:56:08,789] INFO - scvi.dataset.dataset | Downsampling from 46716 to 39851 genes
[2020-06-03 20:56:09,335] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-03 20:56:09,419] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-06-03 20:56:09,883] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-03 20:56:09,964] INFO - scvi.dataset.dataset | Downsampled from 19054 to 19054 cells
GeneExpressionDataset object with n_cells x nb_genes = 19054 x 39851
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'labels', 'batch', 'loca

Concatenate data with highly variable genes

In [0]:
all_dataset = GeneExpressionDataset()

stim_data.subsample_genes(5000)
fs_data.subsample_genes(5000)
all_dataset.populate_from_datasets([stim_data,fs_data])

Create and fit LDVAE model

In [0]:
#Set n_latents to 30
vae = LDVAE(
    all_dataset.nb_genes,
    n_batch=all_dataset.n_batches,
    n_latent=30,
    n_layers_encoder=1,
    n_hidden=128,
    reconstruction_loss="nb",
    latent_distribution="normal",
)

In [0]:
trainer = UnsupervisedTrainer(vae,
                              all_dataset,
                              frequency=1,
                              use_cuda=True,
                              n_epochs_kl_warmup=None
                             )

Train model

In [0]:
n_epochs_all = None

n_epochs = 250 if n_epochs_all is None else n_epochs_all
trainer.train(lr=5e-3, n_epochs=250)

In [0]:
#ELBO plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(trainer.history['elbo_train_set'][10:])
ax.plot(trainer.history['elbo_test_set'][10:])

Save data

In [0]:
torch.save(trainer.model.state_dict(),'harmonization.vae.allgenes.model.pkl')

Load data

In [0]:
trainer.model.load_state_dict(torch.load('harmonization.vae.allgenes.model.pkl'))
#trainer.model.eval()

In [0]:
#Extract latent space
full = trainer.create_posterior(trainer.model, all_dataset, indices=np.arange(len(all_dataset)))
Z_hat = full.sequential().get_latent()[0]
adata = anndata.AnnData(all_dataset.X)
adata.var_names = all_dataset.gene_names
adata.obs_names = list(jelly4stim.obs_names)+list(oj2.obs_names)
adata

Clustering of data

In [0]:
#Use UMAP embedding and Leiden clustering for now
adata.obsm["X_scVI"] = Z_hat
sc.pp.neighbors(adata, use_rep="X_scVI", n_neighbors=1000)
sc.tl.umap(adata)
sc.tl.leiden(adata, key_added="leiden_scVI", resolution=0.8)
adata

In [0]:
sc.pl.umap(adata, color=["leiden_scVI"],color_map='viridis',legend_loc='on data')

In [0]:
#Add labels for experimental origin of each cell
origin_label = list(np.repeat('IEG',len(jelly4stim.obs_names))) + list(np.repeat('FS',len(oj2.obs_names)))
adata.obs['cell_origin'] =  pd.Categorical(origin_label)
adata

Plot latent variables

In [0]:
loadings = vae.get_loadings()
loadings = \
pd.DataFrame.from_records(loadings, index=all_dataset.gene_names,
                                    columns=[f'Z_{i}' for i in range(30)])
loadings.head()

In [0]:
for i, z in enumerate(Z_hat.T):
    adata.obs[f'Z_{i}'] = z

In [0]:
# Plot latent variables
# zs = [f'Z_{i}' for i in range(vae.n_latent)]
# sc.pl.umap(adata, color=zs, ncols=3)

Save adata

In [0]:
# adata.write('scVI_latent30_combo.h5ad')

# #Save obs to combo data, which contains all nonzero ('unfiltered') genes for downstream analysis/visualizations
# combo.obs['leiden_scVI'] = adata.obs['leiden_scVI']
# combo.obs['cell_origin'] = adata.obs['cell_origin']
# combo.obsm['X_scVI'] = adata.obsm['X_scVI']
# combo.obsm['X_umap'] = adata.obsm['X_umap']

# combo.write('bus_stim_fs_latent30_combo.h5ad')